This notebook follows the development of an algorithm for a recommendation engine pursuant to a given movie title.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import helper functions
%run ./support_functions.ipynb

In [58]:
# import project algorithms
%run ./project_algos.ipynb

In [131]:
pd.options.display.max_colwidth = 300

In [4]:
data = pd.read_csv("data_preprocessed_v2.csv")

In [5]:
backup = data.copy()

In [6]:
info(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298048 entries, 0 to 298047
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   uids          298048 non-null  object 
 1   titles        298048 non-null  object 
 2   genres        298048 non-null  object 
 3   ratings       298048 non-null  object 
 4   scores        298048 non-null  float64
 5   votes         298048 non-null  int64  
 6   lengths       298048 non-null  int64  
 7   directors     298048 non-null  object 
 8   stars         298048 non-null  object 
 9   descriptions  298048 non-null  object 
 10  year          298048 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 25.0+ MB


#### Import libraries for text mining

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

Initialize TF-IDF, eliminating English stop words

In [8]:
tfidf = TfidfVectorizer(stop_words="english")

In [10]:
data.shape

(298048, 11)

In [11]:
data

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
0,tt0066026,MASH,"Comedy, Drama, War",R,7.4,67665,116,Robert Altman,"Donald Sutherland, Elliott Gould, Tom Skerritt...",The staff of a Korean War field hospital use h...,1970
1,tt0065988,Little Big Man,"Adventure, Comedy, Drama",PG-13,7.6,32952,139,Arthur Penn,"Dustin Hoffman, Faye Dunaway, Chief Dan George...","Jack Crabb, looking back from extreme old age,...",1970
2,tt0066011,Love Story,"Drama, Romance",PG,6.9,31300,100,Arthur Hiller,"Ali MacGraw, Ryan O'Neal, John Marley, Ray Mil...",A boy and a girl from different backgrounds fa...,1970
3,tt0065134,Two Mules for Sister Sara,"Adventure, Romance, War",GP,7.0,24029,116,Don Siegel,"Clint Eastwood, Shirley MacLaine, Manolo Fábre...",Nun Sara (Shirley MacLaine) is on the run in M...,1970
4,tt0065421,The AristoCats,"Animation, Adventure, Comedy",G,7.1,94523,78,Wolfgang Reitherman,"Phil Harris, Eva Gabor, Sterling Holloway, Sca...","With the help of a smooth talking tomcat, a fa...",1970
...,...,...,...,...,...,...,...,...,...,...,...
298043,tt10481968,The Lion Rock,"Comedy, Drama, Romance",None,-1.0,0,97,Rosario Scandura,unknown,"The Lion Rock is a movie rich with nostalgia, ...",2020
298044,tt13403734,Never There,Drama,None,-1.0,0,90,Peter Morton,"Peter Morton, Makenzie Dozeman, Ronald F.T. Be...",unknown,2020
298045,tt13564676,Petits rêves,unknown,None,-1.0,0,-1,Mohamed Karrat,"Rachid El Ouali, Kamal Haimoud",unknown,2020
298046,tt11924864,In a Good Place Now: The Life & Music of Bobby...,Documentary,None,-1.0,0,-1,David DuBos,unknown,Feature-length documentary exploring the life ...,2020


sklearn generates a runtime error because the matrix is too large for sklearn to work with it, so the data had to be reduced.

In [12]:
data = data.loc[data.votes >= 500]

In [13]:
data.shape

(40439, 11)

In [17]:
data = data.loc[data.descriptions.str.len() > len("unknown")]

In [42]:
data.reset_index(inplace=True, drop=True)

In [43]:
data.shape

(40283, 11)

Generate TF-IDF matrix

In [19]:
matrix = tfidf.fit_transform(data.descriptions)

In [44]:
matrix.shape

(40283, 44741)

Generate cosine similarity matrix

In [21]:
cossim = linear_kernel(matrix, matrix)

Create a reverse index to look up movies

In [45]:
data.titles = data.titles.apply(lambda x: x.title())

In [46]:
movie_index = pd.Series(data.index, index=data.titles).drop_duplicates()

In [83]:
movie_index.shape

(40283,)

In [128]:
data.loc[data.titles == "Two Mules For Sister Sara"].descriptions.values[0]

'Nun Sara (Shirley MacLaine) is on the run in Mexico and is saved from cowboys by Hogan (Clint Eastwood), who is preparing for a future mission to capture a French fort. The pair become good friends, but Sara never does tell him the true reason behind her being outlawed.'

In [170]:
movie_index["Babe"]

9003

In [82]:
movie_index[1:11]

titles
Little Big Man                1
Love Story                    2
Two Mules For Sister Sara     3
The Aristocats                4
Rio Lobo                      5
Five Easy Pieces              6
Catch-22                      7
Kelly'S Heroes                8
The Kremlin Letter            9
Patton                       10
dtype: int64

In [91]:
name = "a league of their own"
name = name.title()
idx = movie_index[name]

In [96]:
similarities = list(enumerate(cossim[idx]))
similarities

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.03074115719224327),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.03294587205339922),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.03164947276765417),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.0),
 (56, 0.0),
 (57, 0.0),
 (58, 0.0),
 (59, 0.0),
 (60, 0.10438522764877017),
 (61, 0.0),
 (62, 0.0),
 (63, 0.0),
 (64, 0.0),
 (65, 0.0),
 (66, 0.0),
 (67, 0.0),
 (68, 0.0),
 (69, 0.0),
 (70, 0.0),
 (71, 0.0),
 (72, 0.0),
 (73, 0.0),
 (74, 0.0),
 (75, 0.0),
 (76, 0.0),
 (77, 0.0),
 (78, 0.0)

In [105]:
simlist = sorted(similarities, reverse=True, key=lambda x: x[1])
simlist

[(7609, 0.9999999999999999),
 (28130, 0.3295633929698433),
 (38452, 0.23465031689306137),
 (21819, 0.22688575334506839),
 (9703, 0.22259638780450197),
 (8721, 0.2213217596474771),
 (2511, 0.21723377804866728),
 (40116, 0.2169222336787644),
 (29758, 0.21359285437808806),
 (8582, 0.21159098271686855),
 (31955, 0.2099149221109246),
 (33222, 0.20209901173318418),
 (13484, 0.19468609514657031),
 (15438, 0.1945882856205219),
 (20294, 0.1923795564068132),
 (20616, 0.18512177777698363),
 (12736, 0.17955163830712542),
 (5871, 0.17864202430827159),
 (35923, 0.16870809333208148),
 (4787, 0.16859826436103129),
 (28691, 0.16698829090507009),
 (35020, 0.1623824759600211),
 (17362, 0.15786976020114973),
 (38686, 0.1571287297038964),
 (32118, 0.15659650347849435),
 (347, 0.15515327364344522),
 (35352, 0.15238840513080476),
 (28359, 0.15236743869884575),
 (28637, 0.1505316843293992),
 (38941, 0.1488252925565492),
 (31978, 0.1474410254946535),
 (29757, 0.14574464169918283),
 (26877, 0.14510820506219674)

In [108]:
indices = [el[0] for el in simlist]
indices

[7609,
 28130,
 38452,
 21819,
 9703,
 8721,
 2511,
 40116,
 29758,
 8582,
 31955,
 33222,
 13484,
 15438,
 20294,
 20616,
 12736,
 5871,
 35923,
 4787,
 28691,
 35020,
 17362,
 38686,
 32118,
 347,
 35352,
 28359,
 28637,
 38941,
 31978,
 29757,
 26877,
 28863,
 32782,
 36711,
 40054,
 12556,
 34719,
 12568,
 13558,
 28758,
 4734,
 4736,
 32650,
 13548,
 8622,
 12947,
 9923,
 26530,
 31426,
 16856,
 21492,
 16675,
 37922,
 34952,
 37134,
 29739,
 32552,
 15883,
 19797,
 39052,
 7647,
 31483,
 1160,
 9561,
 28397,
 36795,
 15304,
 15379,
 8534,
 14032,
 4343,
 38231,
 13564,
 9738,
 12835,
 6861,
 3787,
 37630,
 7757,
 37608,
 22605,
 4380,
 18867,
 20312,
 10923,
 40059,
 26136,
 22256,
 23222,
 31900,
 3669,
 16328,
 24316,
 36519,
 14101,
 33048,
 33946,
 36884,
 20218,
 28915,
 37262,
 12447,
 34535,
 4662,
 39465,
 37482,
 7862,
 13669,
 26637,
 32534,
 29830,
 29953,
 26281,
 9106,
 60,
 12244,
 32397,
 27720,
 35896,
 35061,
 20024,
 19465,
 13024,
 21376,
 4581,
 26034,
 28601,

In [129]:
def recommend_byTitle(name, n_movies=10, short=False):
    name = name.title()
    idx = movie_index[name]
    
    similarities = sorted(list(enumerate(cossim[idx])), reverse=True, key=lambda x: x[1])
    
    simlist = similarities[1:n_movies+1]
    
    indices = [entry[0] for entry in simlist]
    
    df = data.loc[indices]
    
    df = weighted_scoring(df)
    
    df.reset_index(inplace=True)
    df["index"] = list(df.index+1)
    df.drop(["uids","scores", "votes"], axis=1, inplace=True)
    
    mapper = {"index": "Rank", "titles": "Film", "genres":"Genre", "ratings":"Rating",
              "lengths":"Runtime", "directors":"Director", "stars": "Also starring",
              "descriptions": "Description", "year": "Year", "Weighted_score": "Score"}
    
    df = df.rename(columns=mapper)
    if short:
        df = df[["Rank", "Film", "Description"]]
    
    return df

In [130]:
recommend_byTitle("Spider-Man", 5)

,Rank,Film,Genre,Rating,Runtime,Director,Also starring,Description,Year,Score
0,1,Spider-Man: Into The Spider-Verse,"Animation, Action, Adventure",PG,117,Bob Persichetti,"Shameik Moore, Jake Johnson, Hailee Steinfeld, Mahershala Ali","Teen Miles Morales becomes the Spider-Man of his universe, and must join with five spider-powered individuals from other dimensions to stop a threat for all realities.",2018,8.396864
1,2,Spider-Man: Homecoming,"Action, Adventure, Sci-Fi",PG-13,133,Jon Watts,"Tom Holland, Michael Keaton, Robert Downey Jr., Marisa Tomei","Peter Parker balances his life as an ordinary high school student in Queens with his superhero alter-ego Spider-Man, and finds himself on the trail of a new menace prowling the skies of New York City.",2017,7.399615
2,3,The Amazing Spider-Man,"Action, Adventure, Fantasy",PG-13,136,Marc Webb,"Andrew Garfield, Emma Stone, Rhys Ifans, Irrfan Khan","After Peter Parker is bitten by a genetically altered spider, he gains newfound, spider-like powers and ventures out to save the city from the machinations of a mysterious reptilian foe.",2012,6.900534
3,4,Newcastle,"Drama, Romance, Sport",Unrated,107,Dan Castle,"Lachlan Buchanan, Xavier Samuel, Reshad Strik, Kirk Jenkins",Tragedy befalls one of three brothers while on an outback surfing outing with friends.,2008,6.532560
4,5,Kill Order,"Action, Sci-Fi",Not Rated,77,James Mark,"Chris Mark, Jessica Clement, Denis Akiyama, Daniel Park",A troubled high school student discovers the truth behind his hidden abilities.,2017,5.811761


In [120]:
recommend_byTitle("2010")

,Rank,Film,Genre,Rating,Runtime,Director,Also starring,Description,Year,Score
0,1,Fortress Of War,"Action, Drama, History",Not Rated,138,Aleksandr Kott,"Aleksey Kopashov, Andrey Merzlikin, Pavel Derevyanko, Aleksandr Korshunov","A war drama set during the Nazi invasion of the Soviet Union in June 1941, in which Soviet troops held on to a border stronghold for nine days.",2010,7.365523
1,2,Everest,"Action, Adventure, Biography",PG-13,121,Baltasar Kormákur,"Jason Clarke, Ang Phula Sherpa, Thomas M. Wright, Martin Henderson","The story of New Zealand's Robert ""Rob"" Edwin Hall, who on May 10, 1996, together with Scott Fischer, teamed up on a joint expedition to ascend Mount Everest.",2015,7.095196
2,3,Rappakal,"Comedy, Drama",None,137,Kamal,"Mammootty, Nayanthara, Sharada, Balachandra Menon",The story of a mom and her adopted son and how their life changes after a long-awaited reunion of the joint family.,2005,6.543797
3,4,Europa Report,"Action, Drama, Mystery",PG-13,90,Sebastián Cordero,"Sharlto Copley, Michael Nyqvist, Christian Camargo, Embeth Davidtz",An international crew of astronauts undertakes a privately funded mission to search for life on Jupiter's fourth largest moon.,2013,6.396248
4,5,Coldwater,"Drama, Thriller",Not Rated,104,Vincent Grashaw,"P.J. Boudousqué, James C. Burns, Chris Petrovski, Octavius J. Johnson","A teenage boy is sent to a juvenile reform facility in the wilderness. As we learn about the tragic events that sent him there, his struggle becomes one for survival with the inmates, counselors, ...",2013,6.348664
5,6,36 Fillette,"Romance, Drama",None,88,Catherine Breillat,"Delphine Zentout, Etienne Chicot, Olivier Parnière, Jean-Pierre Léaud","A night in the life of a young woman who wishes to learn and experience more of life. The men she meets have as much to learn from her as they can offer her, or more.",1988,6.066383
6,7,West Coast,Drama,None,80,Benjamin Weill,"Devi Couzigou, Victor Le Blond, Mathis Crusson, Sullivan Loyez",Four french teenagers in a small town are sure to be real West Coast gangstas. They gonna live their crazy last expedition. They will grow and learn.,2016,6.003235
7,8,In Tranzit,"Drama, Romance, War",R,113,Tom Roberts,"Vera Farmiga, Thomas Kretschmann, John Malkovich, Daniel Brühl","In 1946, a group of German POWs are mistakenly sent to a Soviet female transit prison camp and must cope with the hostility of the Soviet female inmates and guards, under the orders of cruel camp ...",2008,5.906876
8,9,Fast Food,Comedy,PG-13,92,Michael A. Simpson,"Clark Brandon, Randal Patrick, Tracy Griffith, Michael J. Pollard",What would happen if a sex drug ended up in the Secret Sauce of a new burger joint?,1989,5.576584
9,10,Iron Eagle Ii,"Action, Drama, Romance",PG,105,Sidney J. Furie,"Louis Gossett Jr., Mark Humphrey, Stuart Margolin, Alan Scarfe",General Chappy Sinclair assembles a joint U.S.- Soviet strike team to take out a rogue middle eastern nuclear base.,1988,4.211678


In [121]:
recommend_byTitle("spaceballs")

,Rank,Film,Genre,Rating,Runtime,Director,Also starring,Description,Year,Score
0,1,Star Wars: Episode Iv - A New Hope,"Action, Adventure, Fantasy",PG,121,George Lucas,"Mark Hamill, Harrison Ford, Carrie Fisher, Alec Guinness","Luke Skywalker joins forces with a Jedi Knight, a cocky pilot, a Wookiee and two droids to save the galaxy from the Empire's world-destroying battle station, while also attempting to rescue Prince...",1977,8.598316
1,2,Never Surrender: A Galaxy Quest Documentary,Documentary,None,95,Jack Bennett,"Tim Allen, Greg Berlanti, Elizabeth Cantillon, Linda DeScenna","Filmmakers and stars discuss the filming and social effects of Galaxy Quest, a comedic take-off of Star Trek, with brilliant commentary not only on the Star Trek series but on the real-life actors...",2019,7.153938
2,3,Pegasus,"Comedy, Sport",None,98,Han Han,"Teng Shen, Johnny Huang, Zheng Yin, Benyu Zhang",An old-time racing champion tries to come back to the race track.,2019,6.389100
3,4,Six Pack,"Comedy, Drama, Sport",PG,108,Daniel Petrie,"Kenny Rogers, Diane Lane, Erin Gray, Barry Corbin",A race car driver is driving to a race in a motor home with his race car on a trailer. His car gets stripped of parts. He ends up with 6 orphan kids on his way to the race.,1982,6.138323
4,5,San Andreas,"Action, Adventure, Thriller",PG-13,114,Brad Peyton,"Dwayne Johnson, Carla Gugino, Alexandra Daddario, Colton Haynes","In the aftermath of a massive earthquake in California, a rescue-chopper pilot makes a dangerous journey with his ex-wife across the state in order to rescue his daughter.",2015,6.101927
5,6,Barbie: The Princess & The Popstar,"Animation, Family, Musical",TV-Y,86,Ezekiel Norton,"Kelly Sheridan, Jennifer Waris, Ashleigh Ball, Tiffany Kathryn","When the gracious young Princess, Tori, trades places with her favourite singer, Keira, the girls will have to work together to save the realm of Meribella. Will the Princess and the Pop Star lear...",2012,6.012946
6,7,Sinbad: Beyond The Veil Of Mists,"Action, Animation",PG,85,Evan Ricks,"Alice Amter, Clint Carmichael, Brendan Fraser, Jennifer Hale","The sailor is ""persuaded"" by a Princess to help her rescue her kingdom from something fishy.",2000,5.882935
7,8,Evil Alien Conquerors,"Comedy, Sci-Fi",PG-13,89,Chris Matheson,"Michael Weston, Diedrich Bader, Chris Parnell, Tyler Labine",Inept beings from the planet Kabijj land on Earth and attempt to behead all of its beings and take over the planet.,2003,5.716533
8,9,Arena,"Action, Adventure, Sci-Fi",PG-13,115,Peter Manoogian,"Paul Satterfield, Hamilton Camp, Claudia Christian, Marc Alaimo",A human becomes an unlikely rising star in the biggest fighting tournament in the galaxy that's dominated by alien species.,1989,5.684755
9,10,Chatur Singh Two Star,"Action, Adventure, Comedy",Not Rated,150,Ajay Chandhok,"Sanjay Dutt, Ameesha Patel, Anupam Kher, Satish Kaushik",A bumbling police detective and his sidekick attempt to arrest an assassin.,2011,4.743164


In [89]:
recommend_byTitle("galaxy quest", 5)

,Rank,Film,Genre,Rating,Runtime,Director,Also starring,Description,Year,Score
0,1,Attack The Block,"Action, Adventure, Comedy",R,88,Joe Cornish,"John Boyega, Jodie Whittaker, Alex Esmail, Lee...",A teen gang in South London defend their block...,2011,6.593472
1,2,Returner,"Action, Adventure, Sci-Fi",R,116,Takashi Yamazaki,"Takeshi Kaneshiro, Anne Suzuki, Gorô Kishitani...",A young woman from the future forces a local g...,2002,6.361791
2,3,Six Pack,"Comedy, Drama, Sport",PG,108,Daniel Petrie,"Kenny Rogers, Diane Lane, Erin Gray, Barry Corbin",A race car driver is driving to a race in a mo...,1982,5.983862
3,4,Earth To Echo,"Adventure, Family, Sci-Fi",PG,91,Dave Green,"Teo Halm, Astro, Reese Hartwig, Ella Wahlestedt",After receiving a bizarre series of encrypted ...,2014,5.807237
4,5,The Darkest Hour,"Action, Adventure, Horror",PG-13,89,Chris Gorak,"Emile Hirsch, Olivia Thirlby, Max Minghella, R...","In Moscow, five young people lead the charge a...",2011,4.917634


In [137]:
d = recommend_byTitle("kill order")
d.loc[9].Description

'After 20 years of graduating from high school, Kayhan goes to the reunion-rice-day of school to meet with high school friends whom he has not seen for years. But here comes a bitter truth. ...                See full summary\xa0»'

In [144]:
d

,Rank,Film,Genre,Rating,Runtime,Director,Also starring,Description,Year,Score
0,1,Bang Bang You'Re Dead,"Crime, Drama, Thriller",R,93,Guy Ferland,"Tom Cavanagh, Ben Foster, Randy Harrison, Janel Moloney","Trevor is a troubled high school student, thanks to the effects of bullying. This is the story of his fight to break free.",2002,7.642642
1,2,Promoción Fantasma,"Adventure, Comedy, Fantasy",R,88,Javier Ruiz Caldera,"Raúl Arévalo, Alexandra Jiménez, Javier Bódalo, Anna Castillo",A teacher with paranormal abilities helps a group of ghosts graduate high school.,2012,6.152602
2,3,Chemical Hearts,"Drama, Romance",R,93,Richard Tanne,"Lili Reinhart, Austin Abrams, Sarah Jones, Adhir Kalyan",A high school transfer student finds a new passion when she begins to work on the school's newspaper.,2020,6.144935
3,4,Teen Wolf,"Comedy, Fantasy, Romance",PG,91,Rod Daniel,"Michael J. Fox, James Hampton, Susan Ursitti, Jerry Levine",An ordinary high school student discovers that his family has an unusual pedigree when he finds himself turning into a werewolf.,1985,6.091305
4,5,Teen Witch,"Comedy, Fantasy, Romance",PG-13,90,Dorian Walker,"Robyn Lively, Dan Gauthier, Joshua John Miller, Caren Kaye","High school misfit Louise, at a loss for romance, discovers magical abilities, but the teenage witch finds that she cannot conjure herself true love, so what?",1989,6.056021
5,6,Power Rangers,"Action, Adventure, Sci-Fi",PG-13,124,Dean Israelite,"Dacre Montgomery, Naomi Scott, RJ Cyler, Ludi Lin","A group of high-school students, who are infused with unique superpowers, harness their abilities in order to save the world.",2017,5.897954
6,7,The Brain,"Horror, Sci-Fi",R,94,Ed Hunt,"Tom Bresnahan, Cynthia Preston, David Gale, George Buza","With the help of an alien organism, a man brainwashes audiences through television; a troubled high school student is determined to stop them.",1988,5.433869
7,8,Teeth,"Comedy, Fantasy, Horror",R,94,Mitchell Lichtenstein,"Jess Weixler, John Hensley, Josh Pais, Hale Appleman","Still a stranger to her own body, a high school student discovers she has a physical advantage when she becomes the object of male violence.",2007,5.406773
8,9,Ámbár Tanár Úr,"Romance, Comedy",None,85,Róbert Koltai,"Judit Hernádi, Kata Dobó, Róbert Koltai, Ferenc Kállai","High school teacher falls in love with his ""best"" female student.",1998,5.256974
9,10,Kayhan,Comedy,None,105,Togan Gökbakar,"Sahan Gökbakar, Irfan Kangi, Gökçe Eyüboglu, Vahram Agakay","After 20 years of graduating from high school, Kayhan goes to the reunion-rice-day of school to meet with high school friends whom he has not seen for years. But here comes a bitter truth. ... See full summary »",2018,3.033391


In [146]:
recommend_byTitle("ever after")

,Rank,Film,Genre,Rating,Runtime,Director,Also starring,Description,Year,Score
0,1,Hounddog,"Drama, Music",R,102,Deborah Kampmeier,"Dakota Fanning, David Morse, Piper Laurie, Granoldo Frazier","A drama set in the American South, where a precocious, troubled girl finds a safe haven in the music and movement of Elvis Presley.",2007,6.205433
1,2,Deliver Us From Eva,"Comedy, Drama, Romance",R,105,Gary Hardwick,"Gabrielle Union, LL Cool J, Essence Atkins, Duane Martin",Eva doesn't let her 3 sisters' men push them around. She's always butting in. How to pacify Eva? Find her a man. The men pay a playboy to be that man. The problem is - he falls in love with Eva.,2003,6.039080
2,3,Carriers,"Adventure, Drama, Sci-Fi",PG-13,84,David Pastor,"Chris Pine, Piper Perabo, Lou Taylor Pucci, Emily VanCamp","As a lethal virus spreads globally, four friends seek a reputed plague-free haven. But while avoiding the infected, the travelers turn on one another.",2009,5.992028
3,4,Eri Eri Rema Sabakutani,"Drama, Music, Sci-Fi",None,107,Shinji Aoyama,"Tadanobu Asano, Aoi Miyazaki, Mariko Okada, Masaya Nakahara","A.D. 2015: A virus has been spreading in many cities worldwide. It is a suicidal disease and the virus is infected by pictures. People, once infected, come down with the disease, which ... See full summary »",2005,5.974416
4,5,The Returned,"Action, Adventure, Crime",None,98,Manuel Carballo,"Emily Hampshire, Kris Holden-Ried, Shawn Doyle, Claudia Bassols","In a post-zombie world, where the infected live normal lives, their retroviral drug is running out.",2013,5.871739
5,6,Resident Evil: Afterlife,"Action, Adventure, Horror",R,96,Paul W.S. Anderson,"Milla Jovovich, Ali Larter, Wentworth Miller, Kim Coates","While still out to destroy the evil Umbrella Corporation, Alice joins a group of survivors living in a prison surrounded by the infected who also want to relocate to the mysterious but supposedly unharmed safe haven known only as Arcadia.",2010,5.799038
6,7,I Zombie: The Chronicles Of Pain,"Drama, Horror",None,79,Andrew Parkinson,"Giles Aspen, Ellen Softley, Dean Sipling, Claire Griffin",A young man gets infected and gradually starts turning into a zombie.,1998,5.543194
7,8,Eva & Adam - Fyra Födelsedagar Och Ett Fiasko,"Family, Romance",None,100,Catti Edfeldt,"Ellen Fjæstad, Carl-Robert Holmer-Kårell, Ulrika Bergman, Pablo Martinez","Eva and Adam are 14 and have been together for three years, but Eva is slowly invaded by new thoughts.",2001,5.437860
8,9,Deadlocked,"Horror, Thriller",None,78,Josh Bailey,"Taylor Tunes, Nick Champa, Sophie Giberson, Kyle English","As a zombie virus breaks out, one ragtag group of trapped elevator passengers must pull together for a fighting chance of survival against an infected rider and the clever horde that awaits them outside.",2020,5.318983
9,10,Zombie Strippers!,"Comedy, Horror, Sci-Fi",R,94,Jay Lee,"Jenna Jameson, Robert Englund, Roxy Saint, Penny Vital",A zombie epidemic spreads throughout a strip club in Nebraska.,2008,4.182612


In [145]:
recommend_byTitle("brave")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [148]:
data.loc[data.titles=="Brave"]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
23034,tt1185412,Brave,"Action, Adventure, Drama",Not Rated,4.5,7267,169,Anil Sharma,"Salman Khan, Mithun Chakraborty, Jackie Shroff, Sohail Khan","In early twentieth-century British India, the Pindari leader Prithvi Singh narrates his story to a reporter from the London Times - a story of betrayal and deceit at the hands of the ... See full summary »",2010
25285,tt1217209,Brave,"Animation, Adventure, Comedy",PG,7.1,374949,93,Mark Andrews,"Kelly Macdonald, Billy Connolly, Emma Thompson, Julie Walters","Determined to make her own path in life, Princess Merida (Kelly Macdonald) defies a custom that brings chaos to her kingdom. Granted one wish, Merida must rely on her bravery and her archery skills to undo a beastly curse.",2012


In [183]:
data.loc[movie_index["Brave"]]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
23034,tt1185412,Brave,"Action, Adventure, Drama",Not Rated,4.5,7267,169,Anil Sharma,"Salman Khan, Mithun Chakraborty, Jackie Shroff, Sohail Khan","In early twentieth-century British India, the Pindari leader Prithvi Singh narrates his story to a reporter from the London Times - a story of betrayal and deceit at the hands of the ... See full summary »",2010
25285,tt1217209,Brave,"Animation, Adventure, Comedy",PG,7.1,374949,93,Mark Andrews,"Kelly Macdonald, Billy Connolly, Emma Thompson, Julie Walters","Determined to make her own path in life, Princess Merida (Kelly Macdonald) defies a custom that brings chaos to her kingdom. Granted one wish, Merida must rely on her bravery and her archery skills to undo a beastly curse.",2012


In [149]:
data.loc[data.titles=="Heat"]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
816,tt0068688,Heat,"Comedy, Drama, Romance",R,6.2,1685,102,Paul Morrissey,"Joe Dallesandro, Sylvia Miles, Andrea Feldman, Pat Ast","Spoof of the casual sexual adventures of a one-time child actor in Hollywood. It also involves a bratty on-again, off-again lesbian character.",1972
5191,tt0093164,Heat,"Action, Crime, Drama",R,5.6,1824,101,Dick Richards,"Burt Reynolds, Karen Young, Peter MacNicol, Howard Hesseman","Las Vegas is the backdrop for all the torrid action of heat, as Burt Reynolds plays the soft-hearted bodyguard who's out to protect his friends.",1986
8956,tt0113277,Heat,"Crime, Drama, Thriller",R,8.2,580976,170,Michael Mann,"Al Pacino, Robert De Niro, Val Kilmer, Jon Voight",A group of professional bank robbers start to feel the heat from police when they unknowingly leave a clue at their latest heist.,1995


In [155]:
data.loc[data.titles=="Batman"]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
6284,tt0096895,Batman,"Action, Adventure",PG-13,7.5,339778,126,Tim Burton,"Michael Keaton, Jack Nicholson, Kim Basinger, Robert Wuhl","The Dark Knight of Gotham City begins his war on crime with his first major enemy being Jack Napier, a criminal who becomes the clownishly homicidal Joker.",1989


In [156]:
data.loc[data.titles=="Dracula"]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
2742,tt0079073,Dracula,"Drama, Horror, Romance",R,6.5,9152,109,John Badham,"Frank Langella, Laurence Olivier, Donald Pleasence, Kate Nelligan","In 1913, the charming, seductive and sinister vampire Count Dracula travels to England in search of an immortal bride.",1979


In [157]:
data.loc[data.titles=="Frankenstein"]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
8520,tt0109836,Frankenstein,"Drama, Horror, Romance",R,6.4,51763,123,Kenneth Branagh,"Robert De Niro, Kenneth Branagh, Helena Bonham Carter, Tom Hulce","When the brilliant but unorthodox scientist Dr. Victor Frankenstein rejects the artificial man that he has created, the Creature escapes and later swears revenge.",1994
24011,tt1795369,Frankenstein,"Drama, Sci-Fi",None,8.7,3239,130,Danny Boyle,"Jonny Lee Miller, Benedict Cumberbatch, Andreea Paduraru, Ella Smith","Created by Victor Frankenstein, the one known only as 'The Creature' sets out to discover the world and the meaning of his life.",2011
30675,tt4086032,Frankenstein,"Horror, Sci-Fi, Thriller",R,5.1,2500,89,Bernard Rose,"Xavier Samuel, Carrie-Anne Moss, Danny Huston, Matthew Jacobs",A married couple of scientists create a modern-day monster.,2015


In [158]:
data.loc[data.titles=="Sherlock Holmes"]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
21043,tt0988045,Sherlock Holmes,"Action, Adventure, Mystery",PG-13,7.6,585134,128,Guy Ritchie,"Robert Downey Jr., Jude Law, Rachel McAdams, Mark Strong",Detective Sherlock Holmes and his stalwart partner Watson engage in a battle of wits and brawn with a nemesis whose plot is a threat to all of England.,2009


In [159]:
data.loc[data.titles=="Mulan"]

,uids,titles,genres,ratings,scores,votes,lengths,directors,stars,descriptions,year
10561,tt0120762,Mulan,"Animation, Adventure, Family",G,7.6,258329,88,Tony Bancroft,"Ming-Na Wen, Eddie Murphy, BD Wong, Miguel Ferrer","To save her father from death in the army, a young maiden secretly goes in his place and becomes one of China's greatest heroines in the process.",1998
39382,tt4566758,Mulan,"Action, Adventure, Drama",PG-13,5.6,112759,115,Niki Caro,"Yifei Liu, Donnie Yen, Li Gong, Jet Li",A young Chinese maiden disguises herself as a male warrior in order to save her father.,2020


In [161]:
tmp = data.loc[0].titles

In [164]:
data.loc[data.titles == tmp, "titles"].count()

1

In [168]:
duplicates = []
for title in data.titles:
    counter = data.loc[data.titles==title, "titles"].count()
    if counter > 1:        
        duplicates.append((title, counter))

In [167]:
len(duplicates)

3629

In [169]:
data.to_csv("data_v3.csv", index=False)